# 디스크 컨트롤러

## 문제 설명

하드디스크는 한 번에 하나의 작업만 수행할 수 있습니다. 디스크 컨트롤러를 구현하는 방법은 여러 가지가 있습니다. 가장 일반적인 방법은 요청이 들어온 순서대로 처리하는 것입니다.

예를 들어
```
- 0ms 시점에 3ms가 소요되는 A작업 요청
- 1ms 시점에 9ms가 소요되는 B작업 요청
- 2ms 시점에 6ms가 소요되는 C작업 요청
```
와 같은 요청이 들어왔습니다. 이를 그림으로 표현하면 아래와 같습니다.
![](https://grepp-programmers.s3.amazonaws.com/files/production/b68eb5cec6/38dc6a53-2d21-4c72-90ac-f059729c51d5.png)
한 번에 하나의 요청만을 수행할 수 있기 때문에 각각의 작업을 요청받은 순서대로 처리하면 다음과 같이 처리 됩니다.
![](https://grepp-programmers.s3.amazonaws.com/files/production/5e677b4646/90b91fde-cac4-42c1-98b8-8f8431c52dcf.png)

```
- A: 3ms 시점에 작업 완료 (요청에서 종료까지 : 3ms)
- B: 1ms부터 대기하다가, 3ms 시점에 작업을 시작해서 12ms 시점에 작업 완료(요청에서 종료까지 : 11ms)
- C: 2ms부터 대기하다가, 12ms 시점에 작업을 시작해서 18ms 시점에 작업 완료(요청에서 종료까지 : 16ms)
```
이 때 각 작업의 요청부터 종료까지 걸린 시간의 평균은 10ms(= (3 + 11 + 16) / 3)가 됩니다.

하지만 A → C → B 순서대로 처리하면

![](https://grepp-programmers.s3.amazonaws.com/files/production/9eb7c5a6f1/a6cff04d-86bb-4b5b-98bf-6359158940ac.png)

```
- A: 3ms 시점에 작업 완료(요청에서 종료까지 : 3ms)
- C: 2ms부터 대기하다가, 3ms 시점에 작업을 시작해서 9ms 시점에 작업 완료(요청에서 종료까지 : 7ms)
- B: 1ms부터 대기하다가, 9ms 시점에 작업을 시작해서 18ms 시점에 작업 완료(요청에서 종료까지 : 17ms)
```

이렇게 A → C → B의 순서로 처리하면 각 작업의 요청부터 종료까지 걸린 시간의 평균은 9ms(= (3 + 7 + 17) / 3)가 됩니다.

각 작업에 대해 [작업이 요청되는 시점, 작업의 소요시간]을 담은 2차원 배열 jobs가 매개변수로 주어질 때, 작업의 요청부터 종료까지 걸린 시간의 평균을 가장 줄이는 방법으로 처리하면 평균이 얼마가 되는지 return 하도록 solution 함수를 작성해주세요. (단, 소수점 이하의 수는 버립니다)

## 제한사항

- jobs의 길이는 1 이상 500 이하입니다.
- jobs의 각 행은 하나의 작업에 대한 [작업이 요청되는 시점, 작업의 소요시간] 입니다.
- 각 작업에 대해 작업이 요청되는 시간은 0 이상 1,000 이하입니다.
- 각 작업에 대해 작업의 소요시간은 1 이상 1,000 이하입니다.
- 하드디스크가 작업을 수행하고 있지 않을 때에는 먼저 요청이 들어온 작업부터 처리합니다.

## 입출력 예

|jobs|return|
|---|---|
|[[0, 3], [1, 9], [2, 6]]|9|

### 입출력 예 설명
문제에 주어진 예와 같습니다.

- 0ms 시점에 3ms 걸리는 작업 요청이 들어옵니다.
- 1ms 시점에 9ms 걸리는 작업 요청이 들어옵니다.
- 2ms 시점에 6ms 걸리는 작업 요청이 들어옵니다.

# 풀이 - 1

**CS: SJF가 평균 대기시간이 가장 짧음**
- 짧은 작업을 먼저 처리하면 뒤에 오는 작업들이 빨리 시작될 수 있기 때문
- 긴 작업을 먼저 시작한다고 하면, 뒤에 작업들은 똑같이 그만큼 기다려야함
- 짧은 작업부터 처리하면, 긴 작업이 바로 시작되는 경우보다 더 많은 작업이 일찍 끝나게 되어 다른 작업들이 빠르게 시작할 수 있음
- **BUT** Starvation 발생


In [18]:
import heapq

def solution(jobs):
    n_jobs = len(jobs)
    jobs.sort(key = lambda x: -x[0])
    min_heap = []
    now_time = 0
    full_time_sum = 0
    while jobs or min_heap:
        # 힙에 작업도 없고, 아직 작업이 도착도 안한 시점 
        # -> 오는 작업 끝낸 시점으로 점프
        if not min_heap and now_time <= jobs[-1][0]:
            start, job_time = jobs.pop()
            now_time = start + job_time
            full_time_sum += job_time
            continue
        # 그 시점에서 작업이 힙에 들어가있어야하는 작업이 있으면 힙에 Push
        while jobs and now_time > jobs[-1][0]:
            job = jobs.pop()
            heapq.heappush(min_heap, (job[1], job[0]))
        # 그 힙에서 시간이 가장 짧은 걸 실행 -> 시점 점프
        job_time, start = heapq.heappop(min_heap)
        now_time += job_time
        full_time_sum += now_time - start
        
    return full_time_sum//n_jobs

In [19]:
test_jobs = [[0, 3], [1, 9], [2, 6]]

solution(test_jobs)

9

# 풀이 - 2

(이전 풀이)

- 시점을 모두 고려 => 생각하기는 쉬운듯

In [ ]:
import heapq

def solution(jobs):
    num_of_jobs = len(jobs)
    heapq.heapify(jobs)
    #print(jobs)
    min_heap = [] # 작업 대기 힙
    finish_time = 0
    count = 0 
    t = 0
    while True:
        #print(f'now_t : {t}')
        if min_heap:
            if finish_time <= t:
                #print(f'현재 min_heap 작업 실행 , {min_heap[0]}')
                job_request = heapq.heappop(min_heap)
                request_time = job_request[1]
                working_time = job_request[0]
                finish_time = t + working_time
                count += finish_time - request_time
                #print(f'count : {count}')
        if jobs:
            while t == jobs[0][0]:
                job_request = heapq.heappop(jobs)
                request_time = job_request[0]
                working_time = job_request[1]
                #print(f'현재 작업 요청 들어옴, working_time : {working_time}')
                if finish_time <= t:
                    #print(f'들어온 작업 바로 수행')
                    finish_time = t + working_time
                    count += finish_time - request_time
                else:
                    heapq.heappush(min_heap, [working_time, request_time])
                if not jobs:
                    break
        if t > finish_time and not jobs and not min_heap:
            break
        t += 1
            
                    
    return count//num_of_jobs